# Downloading Exoplanet Data Solutions

***

This notebook contains solutions to the exercises described in the Exercises section of `downloading_exoplanet_data.ipynb`.

In [ ]:
import ast
import sys
import os
import json
import pprint
import urllib.request
from urllib.parse import quote as urlencode

import numpy as np
import requests
from astropy.table import Table
from astroquery.mast import Observations

# Exercise 1
*Use the MAST API to sort Swift data on WASP-12 b by PI and download it.*

We begin with much of the same procedure as the tutorial notebook.

In [ ]:
object_of_interest = 'WASP-12 b'

resolver_request = {'service':'Mast.Name.Lookup',
                     'params':{'input':object_of_interest,
                               'format':'json'},
                     }

# encoding the request as a json string
req_string = json.dumps(resolver_request)
req_string = urlencode(req_string)

In [ ]:
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
object_of_interest = 'WASP-12 b'

resolver_request = {'service':'Mast.Name.Lookup',
                     'params':{'input':object_of_interest,
                               'format':'json'},
                     }

# encoding the request as a json string
req_string = json.dumps(resolver_request)
req_string = urlencode(req_string)

In [ ]:
# retrieve our system's Python version for the request. 
version = ".".join(map(str, sys.version_info[:3]))

# create HTTP Header Variables
headers = {"Content-type": "application/x-www-form-urlencoded",
           "Accept": "text/plain",
           "User-agent":"python-requests/"+version}


In [ ]:
# base API url
request_url='https://mast.stsci.edu/api/v0/invoke'    

# perform the HTTP request
resp = requests.post(request_url, data="request="+req_string, headers=headers)

In [ ]:
resolved_object = resp.json()

pp.pprint(resolved_object)

In [ ]:
obj_ra = resolved_object['resolvedCoordinate'][0]['ra']
obj_dec = resolved_object['resolvedCoordinate'][0]['decl']

obj_ra, obj_dec

In [ ]:
mast_request = {'service':'Mast.Caom.Cone',
                'params':{'ra':obj_ra,
                          'dec':obj_dec,
                          'radius':0.2},
                'format':'json',
                'pagesize':2000,
                'page':1,
                'removenullcolumns':True,
                'removecache':True}


# encoding the request as a json string
req_string = json.dumps(mast_request)
req_string = urlencode(req_string)

# perform the HTTP request
resp = requests.post(request_url, data="request="+req_string, headers=headers)

# decode the HTTP result
mast_data = resp.json()


print(mast_data.keys())
print("Query status:",mast_data['status'])

In [ ]:
mast_data_table = Table()

col_type_list = [(field['name'], field['type']) for field in mast_data['fields']]

for col, atype in col_type_list:
    
    # recast data types
    if atype=="string":
        atype="str"
    if atype=="boolean":
        atype="bool"
    
    # create column in Table corresponding to the mast_data field
    mast_data_table[col] = np.array([x.get(col, None) for x in mast_data['data']],dtype=atype)
    
print(mast_data_table)

In [ ]:
mast_data_table.columns

Instead of next sorting by `t_min`, though, we sort by PI.

In [ ]:
mast_data_table.sort('proposal_pi')

In [ ]:
print(mast_data_table)

In [ ]:
recent_index = -1
interesting_observation = mast_data_table[mast_data_table["obs_collection"] == "SWIFT"][recent_index]
print("Observation:",
      [interesting_observation[x] for x in ['dataproduct_type', 'obs_collection', 'instrument_name']])

In [ ]:
obsid = interesting_observation['obsid']

product_request = {'service':'Mast.Caom.Products',
                  'params':{'obsid':obsid},
                  'format':'json',
                  'pagesize':100,
                  'page':1}   

# encoding the request as a json string
req_string = json.dumps(product_request)
req_string = urlencode(req_string)

# perform the HTTP request
resp = requests.post(request_url, data="request="+req_string, headers=headers)

# dfrecode the HTTP result
obs_products = resp.json()

print("Number of data products:", len(obs_products["data"]))
print("Product information column names:")
pp.pprint(obs_products['fields'])

In [ ]:
pp.pprint([x.get('productType',"") for x in obs_products["data"]])

In [ ]:
sci_prod_arr = [x for x in obs_products['data'] if x.get("productType", None) == 'SCIENCE']
science_products = Table()

col_type_list = [(field['name'], field['type']) for field in obs_products['fields']]

for col, atype in col_type_list:
    
    # recast data types
    if atype=="string":
        atype="str"
    if atype=="boolean":
        atype="bool"
    if atype == "int":
        atype = "float" # array may contain nan values, and they do not exist in numpy integer arrays
        
    # Create column in Table corresponding to the mast_data field
    science_products[col] = np.array([x.get(col,None) for x in sci_prod_arr],dtype=atype)

print("Number of science products:",len(science_products))
print(science_products)

In [ ]:
download_url = 'https://mast.stsci.edu/api/v0.1/Download/file?'

for row in science_products:     

    # Make file path
    out_path = os.path.join("mastFiles", row['obs_collection'], row['obs_id'])
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    out_path = os.path.join(out_path, os.path.basename(row['productFilename']))
        
    # Download the data
    payload = {"uri":row['dataURI']}
    resp = requests.get(download_url, params=payload)
    
    # Save to file
    with open(out_path,'wb') as f:
        f.write(resp.content)
        
    # Check for file 
    if not os.path.isfile(out_path):
        print("ERROR: " + out_path + " failed to download.")
    else:
        print("COMPLETE: ", out_path)

In [ ]:
ls mastFiles/SWIFT/00032242001/

# Exercise 2

*Query the latest light curve of WASP-12 b.*
*Question: Is this too close to the actual tutorial?*

We can fetch light curves from the TESS mission.

In [30]:
search_radius = ".02 deg"

planet_name = 'WASP-12 b'
obs_table = Observations.query_object(planet_name,radius=search_radius)

# only print the first 10 results for clarity
print(obs_table[:10])  

intentType obs_collection provenance_name instrument_name project filters ... dataRights mtFlag srcDen  obsid   distance
---------- -------------- --------------- --------------- ------- ------- ... ---------- ------ ------ -------- --------
   science           TESS            SPOC      Photometer    TESS    TESS ...     PUBLIC  False    nan 27266912      0.0
   science           TESS            SPOC      Photometer    TESS    TESS ...     PUBLIC  False    nan 65432622      0.0
   science           TESS            SPOC      Photometer    TESS    TESS ...     PUBLIC  False    nan 68942182      0.0
   science           TESS            SPOC      Photometer    TESS    TESS ...     PUBLIC  False    nan 71308547      0.0
   science           TESS            SPOC      Photometer    TESS    TESS ...     PUBLIC  False    nan 71741843      0.0
   science           TESS            SPOC      Photometer    TESS    TESS ...     PUBLIC  False    nan 27242856      0.0
   science           TESS       

In [36]:
obs_table

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,distance
str7,str11,str12,str10,str4,str9,str16,str21,str32,str78,float64,float64,str10,str23,int64,float64,float64,float64,float64,float64,str81,float64,str34,str3,int64,str461,str120,str136,str6,bool,float64,str8,float64
science,SPITZER_SHA,SSC Pipeline,IRAC,--,IRAC4,Infrared,WASP12b-ecl-ch24-1,--,000001B3E200,97.594583,29.639722,image,"Wheatley, Peter",2,54768.668948,54768.66908689,10.4,6430.0,9280.0,Searching for hot stratospheres of new transiting exoplanets from the WASP survey,55148.84736098,WASP_C5_50759,y,28566016,POLYGON ICRS 97.64707900 29.67869000 97.63958900 29.59290200 97.54052900 29.59868200 97.54840100 29.68551900 97.64707900 29.67869000,--,https://irsa.ipac.caltech.edu/data/SPITZER/SHA/archive/proc/IRAC013500/r28566016/ch4/bcd/SPITZER_I4_28566016_0001_0000_6_bcd.fits,PUBLIC,False,nan,1692417,0.0
science,SPITZER_SHA,SSC Pipeline,IRAC,--,IRAC2,Infrared,WASP12b-ecl-ch24-1,--,000001B3E200,97.594583,29.639722,image,"Wheatley, Peter",2,54768.668948,54768.66908689,10.4,3990.0,5010.0,Searching for hot stratospheres of new transiting exoplanets from the WASP survey,55148.84736098,WASP_C5_50759,y,28566016,POLYGON ICRS 97.64803700 29.67819700 97.63949100 29.59287700 97.54075000 29.60016300 97.54975300 29.68611500 97.64803700 29.67819700,--,https://irsa.ipac.caltech.edu/data/SPITZER/SHA/archive/proc/IRAC013500/r28566016/ch2/bcd/SPITZER_I2_28566016_0001_0000_6_bcd.fits,PUBLIC,False,nan,1692417,0.0
science,SPITZER_SHA,SSC Pipeline,IRAC,--,IRAC2,Infrared,WASP12b-ecl-ch24-1,--,000001B3E200,97.594583,29.639722,image,"Wheatley, Peter",2,54768.6690955,54768.66923439,10.4,3990.0,5010.0,Searching for hot stratospheres of new transiting exoplanets from the WASP survey,55148.84736098,WASP_C5_50759,y,28566016,POLYGON ICRS 97.64802100 29.67823200 97.63947500 29.59291200 97.54073400 29.60019800 97.54973600 29.68615000 97.64802100 29.67823200,--,https://irsa.ipac.caltech.edu/data/SPITZER/SHA/archive/proc/IRAC013500/r28566016/ch2/bcd/SPITZER_I2_28566016_0002_0000_6_bcd.fits,PUBLIC,False,nan,1692417,0.0
science,SPITZER_SHA,SSC Pipeline,IRAC,--,IRAC4,Infrared,WASP12b-ecl-ch24-1,--,000001B3E200,97.594583,29.639722,image,"Wheatley, Peter",2,54768.6690955,54768.66923439,10.4,6430.0,9280.0,Searching for hot stratospheres of new transiting exoplanets from the WASP survey,55148.84736098,WASP_C5_50759,y,28566016,POLYGON ICRS 97.64706200 29.67872600 97.63957300 29.59293800 97.54051300 29.59871700 97.54838400 29.68555400 97.64706200 29.67872600,--,https://irsa.ipac.caltech.edu/data/SPITZER/SHA/archive/proc/IRAC013500/r28566016/ch4/bcd/SPITZER_I4_28566016_0002_0000_6_bcd.fits,PUBLIC,False,nan,1692417,0.0
science,SPITZER_SHA,SSC Pipeline,IRAC,--,IRAC4,Infrared,WASP12b-ecl-ch24-1,--,000001B3E200,97.594583,29.639722,image,"Wheatley, Peter",2,54768.6692483,54768.66938719,10.4,6430.0,9280.0,Searching for hot stratospheres of new transiting exoplanets from the WASP survey,55148.84736098,WASP_C5_50759,y,28566016,POLYGON ICRS 97.64705400 29.67873900 97.63956500 29.59295100 97.54050500 29.59873000 97.54837600 29.68556700 97.64705400 29.67873900,--,https://irsa.ipac.caltech.edu/data/SPITZER/SHA/archive/proc/IRAC013500/r28566016/ch4/bcd/SPITZER_I4_28566016_0003_0000_6_bcd.fits,PUBLIC,False,nan,1692417,0.0
science,SPITZER_SHA,SSC Pipeline,IRAC,--,IRAC2,Infrared,WASP12b-ecl-ch24-1,--,000001B3E200,97.594583,29.639722,image,"Wheatley, Peter",2,54768.6692483,54768.66938719,10.4,3990.0,5010.0,Searching for hot stratospheres of new transiting exoplanets from the WASP survey,55148.84736098,WASP_C5_50759,y,28566016,POLYGON ICRS 97.64801300 29.67824500 97.63946700 29.59292500 97.54072600 29.60021100 97.54972800 29.68616300 97.6480130

To get the latest results, we sort by t_min.

In [37]:
obs_table.sort('t_min')

However, we want to make sure that we aren't selecting an observation with a `NaN` value for `t_min`.

In [42]:
not_nan_mask = ~np.isnan(obs_table['t_min'])

obs_table = obs_table[not_nan_mask]

In [43]:
obs_table

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,distance
str7,str11,str12,str10,str4,str9,str16,str21,str32,str78,float64,float64,str10,str23,int64,float64,float64,float64,float64,float64,str81,float64,str34,str3,int64,str461,str120,str136,str6,bool,float64,str8,float64
science,SPITZER_SHA,SSC Pipeline,IRAC,--,IRAC4,Infrared,WASP12b-ecl-ch24-1,--,000001B3E200,97.594583,29.639722,image,"Wheatley, Peter",2,54768.668948,54768.66908689,10.4,6430.0,9280.0,Searching for hot stratospheres of new transiting exoplanets from the WASP survey,55148.84736098,WASP_C5_50759,y,28566016,POLYGON ICRS 97.64707900 29.67869000 97.63958900 29.59290200 97.54052900 29.59868200 97.54840100 29.68551900 97.64707900 29.67869000,--,https://irsa.ipac.caltech.edu/data/SPITZER/SHA/archive/proc/IRAC013500/r28566016/ch4/bcd/SPITZER_I4_28566016_0001_0000_6_bcd.fits,PUBLIC,False,nan,1692417,0.0
science,SPITZER_SHA,SSC Pipeline,IRAC,--,IRAC2,Infrared,WASP12b-ecl-ch24-1,--,000001B3E200,97.594583,29.639722,image,"Wheatley, Peter",2,54768.668948,54768.66908689,10.4,3990.0,5010.0,Searching for hot stratospheres of new transiting exoplanets from the WASP survey,55148.84736098,WASP_C5_50759,y,28566016,POLYGON ICRS 97.64803700 29.67819700 97.63949100 29.59287700 97.54075000 29.60016300 97.54975300 29.68611500 97.64803700 29.67819700,--,https://irsa.ipac.caltech.edu/data/SPITZER/SHA/archive/proc/IRAC013500/r28566016/ch2/bcd/SPITZER_I2_28566016_0001_0000_6_bcd.fits,PUBLIC,False,nan,1692417,0.0
science,SPITZER_SHA,SSC Pipeline,IRAC,--,IRAC2,Infrared,WASP12b-ecl-ch24-1,--,000001B3E200,97.594583,29.639722,image,"Wheatley, Peter",2,54768.6690955,54768.66923439,10.4,3990.0,5010.0,Searching for hot stratospheres of new transiting exoplanets from the WASP survey,55148.84736098,WASP_C5_50759,y,28566016,POLYGON ICRS 97.64802100 29.67823200 97.63947500 29.59291200 97.54073400 29.60019800 97.54973600 29.68615000 97.64802100 29.67823200,--,https://irsa.ipac.caltech.edu/data/SPITZER/SHA/archive/proc/IRAC013500/r28566016/ch2/bcd/SPITZER_I2_28566016_0002_0000_6_bcd.fits,PUBLIC,False,nan,1692417,0.0
science,SPITZER_SHA,SSC Pipeline,IRAC,--,IRAC4,Infrared,WASP12b-ecl-ch24-1,--,000001B3E200,97.594583,29.639722,image,"Wheatley, Peter",2,54768.6690955,54768.66923439,10.4,6430.0,9280.0,Searching for hot stratospheres of new transiting exoplanets from the WASP survey,55148.84736098,WASP_C5_50759,y,28566016,POLYGON ICRS 97.64706200 29.67872600 97.63957300 29.59293800 97.54051300 29.59871700 97.54838400 29.68555400 97.64706200 29.67872600,--,https://irsa.ipac.caltech.edu/data/SPITZER/SHA/archive/proc/IRAC013500/r28566016/ch4/bcd/SPITZER_I4_28566016_0002_0000_6_bcd.fits,PUBLIC,False,nan,1692417,0.0
science,SPITZER_SHA,SSC Pipeline,IRAC,--,IRAC4,Infrared,WASP12b-ecl-ch24-1,--,000001B3E200,97.594583,29.639722,image,"Wheatley, Peter",2,54768.6692483,54768.66938719,10.4,6430.0,9280.0,Searching for hot stratospheres of new transiting exoplanets from the WASP survey,55148.84736098,WASP_C5_50759,y,28566016,POLYGON ICRS 97.64705400 29.67873900 97.63956500 29.59295100 97.54050500 29.59873000 97.54837600 29.68556700 97.64705400 29.67873900,--,https://irsa.ipac.caltech.edu/data/SPITZER/SHA/archive/proc/IRAC013500/r28566016/ch4/bcd/SPITZER_I4_28566016_0003_0000_6_bcd.fits,PUBLIC,False,nan,1692417,0.0
science,SPITZER_SHA,SSC Pipeline,IRAC,--,IRAC2,Infrared,WASP12b-ecl-ch24-1,--,000001B3E200,97.594583,29.639722,image,"Wheatley, Peter",2,54768.6692483,54768.66938719,10.4,3990.0,5010.0,Searching for hot stratospheres of new transiting exoplanets from the WASP survey,55148.84736098,WASP_C5_50759,y,28566016,POLYGON ICRS 97.64801300 29.67824500 97.63946700 29.59292500 97.54072600 29.60021100 97.54972800 29.68616300 97.6480130

In [44]:
obs_table_qlp = obs_table[obs_table['project']=='TESS']

In [45]:
recent_index = -1
data_products_by_obs = Observations.get_product_list(obs_table_qlp[recent_index])
print(data_products_by_obs) 

 obsID   obs_collection dataproduct_type                      obs_id                     ...   size   parent_obsid dataRights calib_level
-------- -------------- ---------------- ----------------------------------------------- ... -------- ------------ ---------- -----------
71236476           TESS       timeseries tess2021310001228-s0045-0000000086396382-0216-s ... 18181717     71236476     PUBLIC           3
71236476           TESS       timeseries tess2021310001228-s0045-0000000086396382-0216-s ...   123354     71236476     PUBLIC           3
71236476           TESS       timeseries tess2021310001228-s0045-0000000086396382-0216-s ...  5251286     71236476     PUBLIC           3
71236476           TESS       timeseries tess2021310001228-s0045-0000000086396382-0216-s ...  1273273     71236476     PUBLIC           3
71236476           TESS       timeseries tess2021310001228-s0045-0000000086396382-0216-s ...  3648960     71236476     PUBLIC           3
71236476           TESS       time

In [46]:
data_products_by_obs.columns

<TableColumns names=('obsID','obs_collection','dataproduct_type','obs_id','description','type','dataURI','productType','productGroupDescription','productSubGroupDescription','productDocumentationURL','project','prvversion','proposal_id','productFilename','size','parent_obsid','dataRights','calib_level')>

In [47]:
# select first observation collection
obs_collection = data_products_by_obs['obs_collection'][0]
obs_id = data_products_by_obs['obs_id'][0]

# query and get product list for this collection
single_obs = Observations.query_criteria(obs_collection=obs_collection, obs_id=obs_id)
data_products = Observations.get_product_list(single_obs)

# download data products
manifest = Observations.download_products(data_products, productType="SCIENCE")

# Exercise 3

In [106]:
planet_name = 'BD+47 2936 b'

In [107]:
planet_name_formatted = planet_name.replace(' ', '%20')

request_name_string = f'exoplanets/identifiers/?name={planet_name_formatted}'

In [108]:
request_name_url = "https://exo.mast.stsci.edu/api/v0.1/" + request_name_string
print(request_name_url)

https://exo.mast.stsci.edu/api/v0.1/exoplanets/identifiers/?name=BD+47%202936%20b


In [115]:
names = urllib.request.urlopen(request_name_url).read()
names

b'{"canonicalName":"HAT-P-11 b","starName":"HAT-P-11","ra":297.70891666412354,"dec":48.080294444825924,"planetNames":["HAT-P-11 b","Gaia DR1 2086512223550151936 b","HIP 97657 b","Kepler-3 b","G 208-41 b","BD+47 2936 b","KOI 3 b","KIC 10748390 b","TYC 3561-2092-1 b","NLTT 48335 b","LSPM J1950+4804 b","USNO-B1.0 1380-00392296 b","ASCC 346382 b","GSC 03561-02092 b","2MASS J19505021+4804508 b","HIC 97657 b"],"keplerID":10748390,"keplerTCE":"TCE_1","tessID":28230919,"tessTCE":"TCE_1"}'

Now, we note that the canonical name for this planet is HAT-P-11 b. We use this name for our data request.

In [118]:
planet_name = 'HAT-P-11 b'
planet_name_formatted = planet_name.replace(' ', '%20')

In [119]:
dict_str = names.decode("UTF-8")

In [120]:
dict_str = dict_str.replace('null', 'None')
dict_str

'{"canonicalName":"HAT-P-11 b","starName":"HAT-P-11","ra":297.70891666412354,"dec":48.080294444825924,"planetNames":["HAT-P-11 b","Gaia DR1 2086512223550151936 b","HIP 97657 b","Kepler-3 b","G 208-41 b","BD+47 2936 b","KOI 3 b","KIC 10748390 b","TYC 3561-2092-1 b","NLTT 48335 b","LSPM J1950+4804 b","USNO-B1.0 1380-00392296 b","ASCC 346382 b","GSC 03561-02092 b","2MASS J19505021+4804508 b","HIC 97657 b"],"keplerID":10748390,"keplerTCE":"TCE_1","tessID":28230919,"tessTCE":"TCE_1"}'

In [121]:
name_matches = ast.literal_eval(dict_str)

In [122]:
name_matches

{'canonicalName': 'HAT-P-11 b',
 'starName': 'HAT-P-11',
 'ra': 297.70891666412354,
 'dec': 48.080294444825924,
 'planetNames': ['HAT-P-11 b',
  'Gaia DR1 2086512223550151936 b',
  'HIP 97657 b',
  'Kepler-3 b',
  'G 208-41 b',
  'BD+47 2936 b',
  'KOI 3 b',
  'KIC 10748390 b',
  'TYC 3561-2092-1 b',
  'NLTT 48335 b',
  'LSPM J1950+4804 b',
  'USNO-B1.0 1380-00392296 b',
  'ASCC 346382 b',
  'GSC 03561-02092 b',
  '2MASS J19505021+4804508 b',
  'HIC 97657 b'],
 'keplerID': 10748390,
 'keplerTCE': 'TCE_1',
 'tessID': 28230919,
 'tessTCE': 'TCE_1'}

In [123]:
# construct request string
request_spectra_string = f'spectra/{planet_name_formatted}/filelist/'
request_spectra_url = "https://exo.mast.stsci.edu/api/v0.1/" + request_spectra_string

# send the request
spectra_result = urllib.request.urlopen(request_spectra_url).read()

# decode the result and make it Pythonic
dict_str = spectra_result.decode("UTF-8")
dict_str = dict_str.replace('null', 'None')

# evaluate the result
spectra = ast.literal_eval(dict_str)
spectra

{'filenames': ['HAT-P-11b_transmission_Chachan2019.txt',
  'HAT-P-11b_transmission_Fraine2014.txt']}

In [124]:
# construct request string
filename = spectra['filenames'][0]
request_file_string = f'spectra/{planet_name_formatted}/file/{filename}'
request_file_url = "https://exo.mast.stsci.edu/api/v0.1/" + request_file_string

# send the request
spectra_result = urllib.request.urlopen(request_file_url).read()

# decode the result and make it Pythonic
downloaded_file = spectra_result.decode("UTF-8")
downloaded_file

'# HAT-P-11b transmission spectrum \n# This file contains:\n# HST STIS G430L - GO 14767 (PI Sing, Lopez-Morales) - Chachan et al. 2019, AJ\n# HST STIS G750L - GO 14767 (PI Sing, Lopez-Morales) - Chachan et al. 2019, AJ\n# HST WFC3 G102 - GO 14793 (PI Bean) - Chachan et al. 2019, AJ\n# HST WFC3 G141 - GO 14260 (PI Deming) - Chachan et al. 2019, AJ\n# Spitzer 3.6, 4.5 -  Chachan et al. 2019, AJ\n# Data history:\n# Converted from: Chachan et al. 2019, AJ\n# ----------------------------------------------------------------------------------------\n# Wavelength (microns)  Delta Wavelength (microns)  (Rp/Rs)^2   (Rp/Rs)^2 +/-uncertainty\n# ----------------------------------------------------------------------------------------\n3.734999999999999987e-01 2.750000000000002442e-02 3.350094400000000212e-03 1.354392000000000161e-04\n4.284999999999999920e-01 2.749999999999999667e-02 3.388404099999999843e-03 5.238899999999999596e-05\n4.835000000000000409e-01 2.749999999999999667e-02 3.396558399999999

In [125]:
with open(filename, 'w') as f:
    f.write(downloaded_file)